In [1]:
import duckdb
import httpx
import pathlib

API_URL = "https://saisonmanager.de/api/v2"

leagues_path = "leagues.json"
league_path = "leagues/{league_id}.json"
standings_path = "leagues/{league_id}/table.json"
scorers_path = "leagues/{league_id}/scorer.json"
schedule_path = "leagues/{league_id}/schedule.json"
game_path = "games/{game_id}.json"

conn = duckdb.connect("floorball.db")


def download_json(url: str, path: pathlib.Path = pathlib.Path("data")):
    with httpx.Client(base_url=API_URL) as client:
        content = client.get(url).raise_for_status().content
        out_path = pathlib.Path(path, url)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        with open(out_path, "wb") as json_file:
            json_file.write(content)


def download_league(league_id: int, path: pathlib.Path = pathlib.Path("data")):
    for url in [
        api_path.format(league_id=league_id)
        for api_path in [league_path, standings_path, scorers_path, schedule_path]
    ]:
        download_json(url, path)

    schedule = (
        httpx.get(f"{API_URL}/{schedule_path.format(league_id=league_id)}")
        .raise_for_status()
        .json()
    )

    for url in [game_path.format(game_id=game["game_id"]) for game in schedule]:
        download_json(url, path)


download_json(leagues_path)

In [2]:
selected_leagues = conn.sql(
    """
CREATE OR REPLACE TABLE leagues AS SELECT * FROM 'data/leagues.json';
SELECT id, game_operation, name, season FROM leagues
WHERE
    season >= (SELECT max(season) FROM leagues) - 1    -- only the last two seasons
    AND game_operation = 'Floorball Deutschland'
    AND NOT contains(name, 'Damen')
    AND (
        starts_with(name, '1. FBL') OR starts_with(name, '2. FBL')
    )
ORDER BY season DESC;
"""
).df()
selected_leagues

,id,game_operation,name,season
0,1442,Floorball Deutschland,1. FBL Herren,15
1,1513,Floorball Deutschland,1. FBL Herren - Playoffs,15
2,1514,Floorball Deutschland,1. FBL Herren - Playdowns,15
3,1444,Floorball Deutschland,2. FBL Herren Nord/West,15
4,1445,Floorball Deutschland,2. FBL Herren Süd/West,15
5,1446,Floorball Deutschland,2. FBL Herren Ost,15
6,1326,Floorball Deutschland,1. FBL Herren - Playoffs,14
7,1330,Floorball Deutschland,1. FBL Herren - Playdowns,14
8,1204,Floorball Deutschland,1. FBL Herren,14
9,1329,Floorball Deutschland,2. FBL Herren - Playoffs,14


In [3]:
for league_id in selected_leagues["id"]:
    download_league(league_id=league_id)

In [38]:
conn.sql(
    """
CREATE OR REPLACE TABLE games AS
    SELECT
        id,
        league_id,
        game_number,
        date,
        start_time,
        actual_start_time,
        game_status,
        home_team_id,
        home_team_name,
        guest_team_id,
        guest_team_name,
        ended,
        result.home_goals AS home_goals,
        result.home_goals_period AS home_goals_period,
        result.guest_goals AS guest_goals,
        result.guest_goals_period AS guest_goals_period,
        result.forfait AS forfait,
        result.overtime AS overtime,
        arena,
        events,
        referees,
        list_transform(players.home,
            player -> {
                id: player.player_id,
                number: player.trikot_number,
                lastname: player.player_name,
                firstname: player.player_firstname,
                goalkeeper: coalesce(player.goalkeeper, false),
                captain: coalesce(player.captain, false)
            }
        ) AS players_home,
        list_transform(players.guest,
            player -> {
                id: player.player_id,
                number: player.trikot_number,
                lastname: player.player_name,
                firstname: player.player_firstname,
                goalkeeper: coalesce(player.goalkeeper, false),
                captain: coalesce(player.captain, false)
            }
        ) AS players_guest
FROM read_json_auto('data/games/*.json', union_by_name = true);
DESCRIBE games;
"""
).df()

,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,NaN,NaN,NaN
1,league_id,BIGINT,YES,NaN,NaN,NaN
2,game_number,BIGINT,YES,NaN,NaN,NaN
3,date,VARCHAR,YES,NaN,NaN,NaN
4,start_time,VARCHAR,YES,NaN,NaN,NaN
5,actual_start_time,VARCHAR,YES,NaN,NaN,NaN
6,game_status,VARCHAR,YES,NaN,NaN,NaN
7,home_team_id,BIGINT,YES,NaN,NaN,NaN
8,home_team_name,VARCHAR,YES,NaN,NaN,NaN
9,guest_team_id,BIGINT,YES,NaN,NaN,NaN


In [21]:
conn.sql(
    """
CREATE OR REPLACE TABLE teams AS SELECT
    team_id AS id,
    team_name AS name,
FROM 'data/leagues/*/table.json';
SELECT * FROM teams;
"""
).df()

,id,name
0,5064,FC Stern München
1,5068,TSV Tollwut Ebersgöns
2,5065,Frankfurt Falcons
3,5062,ESV Ingolstadt Schanzer Ducks
4,5066,TSG Erlensee
...,...,...
93,5068,TSV Tollwut Ebersgöns
94,5052,Unihockey Igels Dresden
95,5049,SSF Dragons Bonn
96,5048,Red Devils Wernigerode


In [34]:
conn.sql(
    """
CREATE OR REPLACE TABLE players AS SELECT
    player.id AS id,
    team_id,
    player.lastname AS lastname,
    player.firstname AS firstname
FROM (
    SELECT
        unnest(players_home) AS player,
        home_team_id AS team_id
    FROM games
    UNION
    SELECT
        unnest(players_guest) AS player,
        guest_team_id AS team_id
    FROM games
);
SELECT * FROM players;
"""
).df()

,id,team_id,lastname,firstname
0,2340,5049,Knoop,Jan Alexander
1,9779,5049,Klapkarek,Fynn-Lennart
2,1094,5049,Weißkirchen,Florian
3,14200,5049,Weh,Johannes
4,1171,5049,Leyhe,Levin
...,...,...,...,...
2210,17622,5046,Müller,Valentin
2211,16435,5954,Bartoniczek,Jonathan
2212,79,5055,Gayk,Daniel
2213,9094,5988,Thiele,Raphael


In [42]:
conn.sql(
    """
CREATE OR REPLACE TABLE events AS SELECT
    game_id,
    event.event_id AS id,
    event.event_type AS type,
    CASE WHEN event.event_team = 'home' THEN home_team_id ELSE guest_team_id END AS team_id,
    event.period AS period,
    event.home_goals AS home_goals,
    event.guest_goals AS guest_goals,
    make_time(
        0,
        extract('minute' FROM try_strptime(event.time, '%M:%S')),
        extract('second' FROM try_strptime(event.time, '%M:%S'))
    ) AS time,
    event.penalty_type AS penalty_type,
    event.penalty_reason AS penalty_reason,
    event.penalty_reason_string AS penalty_reason_name,
    CASE WHEN event.event_team = 'home' THEN home_players[event.number][1].id ELSE guest_players[event.number][1].id END AS player_id,
    CASE WHEN event.event_team = 'home' THEN home_players[event.assist][1].id ELSE guest_players[event.assist][1].id END AS assist_player_id,
FROM (
    SELECT
        id AS game_id,
        home_team_id,
        guest_team_id,
        map_from_entries(
            list_transform(players_home, player -> (player.number, player))
        ) AS home_players,
        map_from_entries(
            list_transform(players_guest, player -> (player.number, player))
        ) AS guest_players,
        unnest(events) AS event,
    FROM games
    WHERE ended
);
SELECT * FROM events;
"""
)

┌─────────┬───────┬─────────┬─────────┬───┬────────────────┬─────────────────────┬───────────┬──────────────────┐
│ game_id │  id   │  type   │ team_id │ … │ penalty_reason │ penalty_reason_name │ player_id │ assist_player_id │
│  int64  │ int64 │ varchar │  int64  │   │     int64      │       varchar       │   int64   │      int64       │
├─────────┼───────┼─────────┼─────────┼───┼────────────────┼─────────────────────┼───────────┼──────────────────┤
│   27619 │     1 │ penalty │    5042 │ … │            920 │ Handspiel           │      6030 │             NULL │
│   27619 │     2 │ goal    │    5042 │ … │           NULL │ NULL                │      5512 │            13826 │
│   27619 │     3 │ goal    │    5042 │ … │           NULL │ NULL                │      4637 │            13826 │
│   27619 │     4 │ goal    │    5049 │ … │           NULL │ NULL                │     21150 │             1094 │
│   27619 │     5 │ goal    │    5049 │ … │           NULL │ NULL                │     1